<a href="https://colab.research.google.com/github/hand-e-fr/OpenHosta/blob/doc/docs/openhosta_mistral_small.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OpenHosta basic example with a Local Mistral-Small

This colab demonstrate simple use cases of OpenHosta. If you do not have an OpenAI (or other) API KEY, you can run the first part **Install a local Phi-4**. Otherwise, directly jump to step 2: **Basic Usage**.

## Install a local Phi-4

In [1]:
!apt install -y screen

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Suggested packages:
  byobu | screenie | iselect ncurses-term
The following NEW packages will be installed:
  screen
0 upgraded, 1 newly installed, 0 to remove and 18 not upgraded.
Need to get 672 kB of archives.
After this operation, 1,029 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 screen amd64 4.9.0-1 [672 kB]
Fetched 672 kB in 1s (937 kB/s)
Selecting previously unselected package screen.
(Reading database ... 124950 files and directories currently installed.)
Preparing to unpack .../screen_4.9.0-1_amd64.deb ...
Unpacking screen (4.9.0-1) ...
Setting up screen (4.9.0-1) ...
Processing triggers for man-db (2.10.2-1) ...


In [2]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [3]:
!screen -dmS ollama ollama serve

In [4]:
# Can take 5 minutes
!ollama run hf.co/bartowski/Mistral-Small-24B-Instruct-2501-GGUF:Q4_K_S hello --verbose  2>&1 | grep -E "([^0]0%|Bonjour|:)"

pulling 919a8b31c63c...   0% ▕                ▏    0 B/ 13 GB                  pulling manifest 
pulling 919a8b31c63c...   0% ▕                ▏    0 B/ 13 GB                  pulling manifest 
pulling 919a8b31c63c...   0% ▕                ▏ 262 KB/ 13 GB                  pulling manifest 
pulling 919a8b31c63c...   0% ▕                ▏  33 MB/ 13 GB                  pulling manifest 
pulling 919a8b31c63c...   0% ▕                ▏  64 MB/ 13 GB                  pulling manifest 
pulling 919a8b31c63c...  10% ▕█               ▏ 1.3 GB/ 13 GB  201 MB/s    1m0spulling manifest 
pulling 919a8b31c63c...  10% ▕█               ▏ 1.3 GB/ 13 GB  201 MB/s    1m0spulling manifest 
pulling 919a8b31c63c...  10% ▕█               ▏ 1.3 GB/ 13 GB  201 MB/s    1m0spulling manifest 
pulling 919a8b31c63c...  10% ▕█               ▏ 1.4 GB/ 13 GB  201 MB/s    1m0spulling manifest 
pulling 919a8b31c63c...  10% ▕█               ▏ 1.4 GB/ 13 GB  201 MB/s    1m0spulling manifest 
pulling 919a8b31c63c...  20% ▕

In [5]:
!pip install OpenHosta

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 3.4 MB/s eta 0:00:00


## Basic Usage of OpenHosta

### Configure the LLM that you want to use

In [6]:
from OpenHosta import config

# Use Mistral-Small-24B (2025 Jannuary) through ollama
my_model=config.Model(
    base_url="http://localhost:11434/v1/chat/completions",
    model="hf.co/bartowski/Mistral-Small-24B-Instruct-2501-GGUF:Q4_K_S", api_key="none", timeout=120
)
config.set_default_model(my_model)


### Emulate functions using the seleted LLM

In [7]:
from OpenHosta import emulate

In [15]:
def translate(text:str, language:str)->str:
    """
    This function translates the text in the “text” parameter into the language specified in the “language” parameter.
    """
    return emulate()

result = translate("Hello World!", "French")
print(result)

Bonjour le monde!


In [16]:
def find_name_age(sentence:str, id:dict)->dict:
    """
    This function find in a text the name and the age of a personn.

    Args:
        sentence: The text in which to search for information
        id: The dictionary to fill in with information

    Return:
        A dictionary identical to the one passed in parameter, but filled with the information.
        If the information is not found, fill with the values with “None”.
    """
    return emulate(model=my_model)

return_dict = {"name": "", "age": ""}
result = find_name_age("Hello, I'm John Wick, i'm 30 and I live in New York", return_dict)
print(result)

{'name': 'John Wick', 'age': '30'}


### Specify advanced return types

In [13]:
from typing import Dict, Tuple, List

def analyze_text(text: str) -> Dict[str, List[Tuple[int, str]]]:
    """Analyze text to map each word to a list of tuples containing word length and word."""
    return emulate()

# Example usage
analysis = analyze_text("Hello, World!")

print(analysis)
print(type(analysis))

{'Hello': [[5, 'Hello']], 'World!': [7, 'World!']}
<class 'dict'>


### Specify pydantic return strucures

OpenHosta is compatible with pydantic. You can specify pydantic input and output types and OpenHosta will propagate schema and Field documentation to the LLM.

In [17]:
!pip install pydantic

In [18]:
from pydantic import BaseModel, Field

class Personn(BaseModel):
    name: str = Field(..., description = "The full name")
    age: int

def find_name_age_pydantic(sentence:str)->Personn:
    """
    This function find in a text the name and the age of a personn.

    Args:
        sentence: The text in which to search for information

    Return:
        A Pydantic model, but filled with the information.
        If the information is not found, fill with the values with “None”.
    """
    return emulate()

result = find_name_age_pydantic("Luke Skywalker is very surprising: he's only 27 when he becomes a Jedi.")
print(result)

name='Luke Skywalker' age=27


### Limitations

The emulation is limited by the LLM capabilities. Try to have it count r in strawberrry and you will get into troubles ;-).
Make sure the LLM is capable and not alucinating before you implement an emulated function.

In [19]:
def find_occurence_of_a_word(word :str, text: str) -> int:
    """
    This function takes a word and a text and returns
    the number of times the word appears in the text.
    """
    return emulate()

find_occurence_of_a_word("Hello", "Hello World Hello!")


2